In [ ]:
import math
import numpy as np
from scipy import signal
from scipy.fftpack import fft
from skimage.util import view_as_windows
from sklearn.decomposition import PCA


# Estimation of Slow and Diff matrix
# ======================================================================
def diff_matrix_estimation2(fast, frames_per_second):
    avTimeSec = 2

    factor = 1 - math.exp((-1/frames_per_second) / avTimeSec)
    newSlow = fast[0].reshape((1,10))
    vAvMag = 10**(newSlow/20)

    for f in range(0,fast.shape[0]):
        vMag = 10**(fast[f]/20)
        vAvMag = np.multiply(vMag,factor) + np.multiply(vAvMag,(1 - factor))
        vAvdB = 20*np.log10(vAvMag)
        if f == 0:
            newSlow = vAvdB
        else:
            newSlow = np.concatenate((newSlow, vAvdB))

    newDiff = fast - newSlow

    return newDiff


def zcr(x, y):
    """@brief zero crossing detector"""
    return x[np.diff(np.sign(y)) != 0]


def window_and_pca2(diff_matrix, zc1 = 0, zc2 = 0, n = 20):
    # n = 20 -> 30frames
    while (zc1 < 3 or zc2 < 3) and (n > 0):
        s = diff_matrix.shape[0]//n
        step = 1

        wind0 = view_as_windows(diff_matrix, (s, 1), step=(step,1))
        diff_matrix_reshape = wind0.reshape((wind0.shape[0]*10, s))
        # PCA
        pca = PCA()
        pca.fit(diff_matrix_reshape)

        pc1 = signal.detrend(pca.components_[0], type='constant')
        x = np.linspace(0,s-1,s-1)
        zc1 = len(zcr(x, pc1))
        zc2 = len(zcr(x, pc1))
        n -= 5
    return pca, s, diff_matrix_reshape


def freq_per_pc2(pca, window, delta_time, diff_matrix, diff_matrix_reshape):
    # Get frequency per PC
    freq_pc = []
    for k in range(pca.n_components_):
        # FFT for each PC
        N = window*16
        pc = signal.detrend(pca.components_[k], type='constant')
        yf = fft(pc, N)
        freq = np.linspace(0.0, 1.0/(2.0*delta_time), N//2)
        PSD = 2.0/N * np.abs(yf[:N//2])

        peaks, _ = signal.find_peaks(PSD)
        freq_fn = freq[peaks]*60

        ener_max = max(PSD[peaks])
        freq_max = freq_fn[np.where(PSD[peaks]== ener_max)[0][0]]

        item_in_freq_fn = set(freq_fn)

        # Get harmonic indices
        indh = []
        for x in freq_fn:
            harm = 10
            harms = []
            count1 = 2
            count2 = 2
            while harm < 150:
                if harm < 2:
                    harm = x * count2
                    count2 += 1
                    harms.append(harm)
                else:
                    harm = x / count1
                    count1 += 1
                    harms.append(harm)
            count3 = 0
            indf = []
            if x in item_in_freq_fn:
                if round(x,2) in list(map(lambda x: round(x,2), harms)):
                    indf.append(count3)
                count3 += 1
            indh.append(indf)

        #Add harmonic energy
        all_ener_sum = []
        for ih in range(len(indh)):
            if len(indh[ih]) != 0:
                ener_sum = np.sqrt(sum(np.array(list(np.array(PSD[peaks].tolist())[indh[ih]]))**2))
            else:
                ener_sum = 0
            all_ener_sum.append(ener_sum)
        if max(all_ener_sum) > ener_max and k > 3:
            new_f = freq_fn[min(indh[all_ener_sum.index(max(all_ener_sum))])]
            freq_max = new_f
        freq_pc.append(round(freq_max, 3))
    #frequencies
    freq_pc = np.array(freq_pc)
    scores = pca.fit_transform(diff_matrix_reshape)

    arr_freq = (np.argsort(np.abs(scores)))[:,-1]
    arr_freq = freq_pc[arr_freq]
    arr_freq = arr_freq.reshape((arr_freq.shape[0]//10, 10))
    for _ in range(diff_matrix.shape[0]-arr_freq.shape[0]):
        arr_freq = np.concatenate((arr_freq, arr_freq[-1,:].reshape((1,10))))

    return arr_freq, window


def trus_metric_curve():
    tm_values = np.linspace(0, 70, 33)
    tm_values = np.concatenate((tm_values, np.linspace(71, 100, 67)))

    return tm_values


def bpm_compute2(window, arr_freq):
    frames = np.linspace(0,arr_freq.shape[0],arr_freq.shape[0]).tolist()
    bpm = []
    trust_perc = []

    tm_values = trus_metric_curve()

    for j in range(1,window):
        band = 0
        a = arr_freq[:j, band]
        unique, counts = np.unique(a, return_counts=True)
        dict1 = dict(zip(unique, counts))

        for band in range(1,10):
            a = arr_freq[:j, band]
            unique, counts = np.unique(a, return_counts=True)
            dict2 = dict(zip(unique, counts))
            result = {
                key: dict1.get(key, 0) + dict2.get(key, 0) for key in set(dict1) | set(dict2)
            }
            dict1 = result

        freq_total = max(result, key=result.get)
        perc_ini = (result[freq_total]/(j*10))*100
        perc = tm_values[round(perc_ini)-1]

        bpm.append(freq_total)
        trust_perc.append(perc)

    for l in range(arr_freq.shape[0]-window+1):
        band = 0
        a = arr_freq[l:l+window, band]
        unique, counts = np.unique(a, return_counts=True)
        dict1 = dict(zip(unique, counts))

        for band in range(1,10):
            a = arr_freq[l:l+window, band]
            unique, counts = np.unique(a, return_counts=True)
            dict2 = dict(zip(unique, counts))
            result = {
                key: dict1.get(key, 0) + dict2.get(key, 0) for key in set(dict1) | set(dict2)
            }
            dict1 = result

        freq_total = max(result, key=result.get)
        perc_ini = (result[freq_total]/(window*10))*100
        perc = tm_values[round(perc_ini)-1]

        bpm.append(freq_total)
        trust_perc.append(perc)

    return bpm, frames, trust_perc


##------------------------------------------------------------------------------
def python_main(matrix, frames_per_second):
    delta_time = 1 / frames_per_second

    ### Compute Respiratory Rate
    ## Compute difference matrix
    diff_matrix = diff_matrix_estimation2(matrix, frames_per_second)
    ## Get pca, window size, and diff matrix with new shape
    pca, window, diff_matrix_reshape = window_and_pca2(diff_matrix, zc1 = 0, zc2 = 0, n = 20)
    ## Get frequencies array and window size
    arr_freq, window = freq_per_pc2(pca, window, delta_time, diff_matrix, diff_matrix_reshape)
    ## Get bpm, frames, and confidence percentage
    bpm, frames, trust_perc = bpm_compute2(window, arr_freq)

    return bpm, frames, trust_perc


# Tensorflow -----------------------------------------------------------------------------------------

In [ ]:
tf.experimental.numpy.diff(tf.math.sign([-5, 5]))

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>

In [ ]:
tf.get_static_value(tf.experimental.numpy.diff(tf.math.sign([-5, 5])))

array([2], dtype=int32)

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
len((600//tf.Variable(20, trainable=False, dtype=tf.float64), 1))

2

In [ ]:
g=tf.constant([[2,7,6,8], [3,4,9,3], [3,4,0,2]], dtype=tf.float64)
h=tf.constant([[2,3,2,7,0], [4,8,3,1,0], [4,2,7,9,4], [4,9,4,8,3], [9,3,8,3,1]], dtype=tf.float64)

In [ ]:
h[0:g, 1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 4], dtype=int32)>

In [ ]:
h[0:2, 0]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 4], dtype=int32)>

In [ ]:
s = 600//tf.Variable(20, trainable=False, dtype=tf.float64)
s

<tf.Tensor: shape=(), dtype=float64, numpy=30.0>

In [ ]:
sorted_eigenvectors = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
sorted_eigenvectors.write(0, tf.constant([-1,2,3,4], dtype=tf.float64))
sorted_eigenvectors.write(1, tf.constant([2,-3,4,5], dtype=tf.float64))
sorted_eigenvectors.write(2, tf.constant([3,4,5,-6], dtype=tf.float64))
#sorted_eigenvectors = sorted_eigenvectors.stack()

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3257, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3473, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)  File "<ipython-input-17-cbd0a3e9bbcc>", line 2, in <cell line: 2>
    sorted_eigenvectors.write(0, tf.constant([-1,2,3,4], dtype=tf.float64))  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/tf_should_use.py", line 243, in wrapped
    

In [ ]:
sorted_eigenvectors[1]

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 2., -3.,  4.,  5.])>

In [ ]:
N = tf.math.multiply(tf.constant(30, dtype=tf.int32), 16)

In [ ]:
2.0/480 * np.abs(sorted_eigenvectors[:240])

array([[0.00416667, 0.00833333, 0.0125    , 0.01666667],
       [0.00833333, 0.0125    , 0.01666667, 0.02083333],
       [0.0125    , 0.01666667, 0.02083333, 0.025     ]])

In [ ]:
factor = 2.0 / tf.cast(N, tf.float64)

In [ ]:
tf.math.multiply(factor, tf.math.abs(sorted_eigenvectors[:tf.math.floordiv(N, 2)]))

<tf.Tensor: shape=(3, 4), dtype=float64, numpy=
array([[0.00416667, 0.00833333, 0.0125    , 0.01666667],
       [0.00833333, 0.0125    , 0.01666667, 0.02083333],
       [0.0125    , 0.01666667, 0.02083333, 0.025     ]])>

In [ ]:
tf.math.floordiv(600, 2)

<tf.Tensor: shape=(), dtype=int32, numpy=300>

In [ ]:
d=tf.Variable(tf.experimental.numpy.empty(tf.math.floordiv(600, 2), dtype=tf.int32), trainable=False)
d

<tf.Variable 'Variable:0' shape=(300,) dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
d[tf.Variable(1)].assign(tf.Variable(5)-1)

<tf.Variable 'UnreadVariable' shape=(300,) dtype=int32, numpy=
array([0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
d[tf.Variable(3)] + d[tf.Variable(6)] // 2

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [ ]:
linspace_num = tf.math.floordiv(480, 2)
linspace_stop = 1.0 / tf.math.multiply(2.0, 1/tf.constant(10, dtype=tf.float64))
freq = tf.linspace(tf.constant(0, dtype=tf.float64), linspace_stop, linspace_num)

In [ ]:
np.where(freq.numpy()[d.numpy()[:5]]== max(freq.numpy()[d.numpy()[:5]]))

(array([1, 4]),)

In [ ]:
freq_fn = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
freq_fn.write(0, [1,6])
for idx in tf.range(1, d[:5].shape[0]):
    freq_fn.write(idx, [2,5])

In [ ]:
freq_fn = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=True)
freq_fn = freq_fn.write(0, tf.transpose(tf.constant([1,6], dtype=tf.float64)))
for idx in tf.range(1, d[:5].shape[0]):
    freq_fn = freq_fn.write(idx, tf.transpose(tf.constant([2,5], dtype=tf.float64)))

In [ ]:
f= tf.Variable([], shape=tf.TensorShape(None), trainable=False, dtype=tf.float64)

In [ ]:
f[0]

<tf.Tensor: shape=(), dtype=float64, numpy=1.0>

In [ ]:
freq_fn = freq_fn.write(0, tf.transpose(tf.constant([8,4], dtype=tf.float64)))
for idx in tf.range(1, d[:5].shape[0]):
    freq_fn = freq_fn.write(idx, tf.transpose(tf.constant([12,25], dtype=tf.float64)))

In [ ]:
tf.math.reduce_max(freq_fn[:,1])

<tf.Tensor: shape=(), dtype=float64, numpy=6.0>

In [ ]:
tf.math.sqrt(tf.math.reduce_sum(tf.math.pow(freq_fn, 2)))

<tf.Tensor: shape=(), dtype=float64, numpy=7.100653869655707>

In [ ]:
tf.math.reduce_max(freq_fn)

<tf.Tensor: shape=(), dtype=float64, numpy=5.02092050209205>

In [ ]:
tf.where(tf.math.equal(freq_fn, 0))

<tf.Tensor: shape=(0, 2), dtype=int64, numpy=array([], shape=(0, 2), dtype=int64)>

In [ ]:
g=tf.constant([[2,3,4,5],[2,3,4,5]], dtype=tf.float64)
f=tf.constant([[1],[2],[3],[4]], dtype=tf.float64)
#tf.tensordot(g,f, axes=1)

In [ ]:
print(tf.shape(f))
print(tf.shape(g))
@tf.function
def test(g, f):
    tf.print(tf.shape(f))
    tf.print(tf.shape(g))
    scores = tf.linalg.matmul(g,f)
    tf.print(scores)
    #tf.print(tf.argsort(tf.math.abs(scores))[:,-1])

test(g,f)

tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([2 4], shape=(2,), dtype=int32)
[4 1]
[2 4]
[[40]
 [40]]


In [ ]:
g[f[0]]

TypeError: ignored

In [ ]:
tf.unique_with_counts(g)

UniqueWithCounts(y=<tf.Tensor: shape=(4,), dtype=float64, numpy=array([2., 3., 4., 5.])>, idx=<tf.Tensor: shape=(8,), dtype=int32, numpy=array([0, 1, 2, 3, 1, 1, 2, 3], dtype=int32)>, count=<tf.Tensor: shape=(4,), dtype=int32, numpy=array([1, 3, 2, 2], dtype=int32)>)

In [ ]:
tf.concat([[g], [g]], 0).shape

TensorShape([2, 8])

In [ ]:
tf.shape(g)[0]

<tf.Tensor: shape=(), dtype=int32, numpy=3>

In [ ]:
f=tf.constant([[[1,3],[2,4]],[[3,5],[4,6]]], dtype=tf.float64)

In [ ]:
tf.math.subtract(g.shape, tf.TensorShape((2,3)))

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 1], dtype=int32)>

In [ ]:
[-1] + g.shape.as_list()[1:]

[-1, 4]

In [ ]:
f.numpy()

array([[[1., 3.],
        [2., 4.]],

       [[3., 5.],
        [4., 6.]]])

In [ ]:
g[1:]

<tf.Tensor: shape=(2, 4), dtype=float64, numpy=
array([[3., 4., 9., 3.],
       [3., 4., 0., 2.]])>

In [ ]:
g[:-1]

<tf.Tensor: shape=(2, 4), dtype=float64, numpy=
array([[2., 7., 6., 8.],
       [3., 4., 9., 3.]])>

In [ ]:
g[1:]-g[:-1]

<tf.Tensor: shape=(2, 4), dtype=float64, numpy=
array([[ 1., -3.,  3., -5.],
       [ 0.,  0., -9., -1.]])>

In [ ]:
g

<tf.Tensor: shape=(3, 4), dtype=float64, numpy=
array([[2., 7., 6., 8.],
       [3., 4., 9., 3.],
       [3., 4., 0., 2.]])>

In [ ]:
g[:,1:]-g[:,:-1]

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[ 5., -1.,  2.],
       [ 1.,  5., -6.],
       [ 1., -4.,  2.]])>

In [ ]:
tf.math.not_equal(g[:,1:]-g[:,:-1], 0)

<tf.Tensor: shape=(3, 3), dtype=bool, numpy=
array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])>

In [ ]:
tf.where(tf.math.not_equal(g[:,1:]-g[:,:-1], 0))

<tf.Tensor: shape=(9, 2), dtype=int64, numpy=
array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2]])>

In [ ]:
tf.TensorShape((1,10))

TensorShape([1, 10])

In [ ]:
tf.linspace(tf.constant(0, dtype=tf.int32), 0, 10)[0]

<tf.Tensor: shape=(), dtype=float64, numpy=0.0>

In [ ]:
twenty_tensor = tf.constant(20, dtype=tf.float64)
five_tensor = tf.constant([5, 2], dtype=tf.float64)
tf.math.pow(tf.constant(10, dtype=tf.float64), tf.math.divide(five_tensor, twenty_tensor))

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([1.77827941, 1.25892541])>

In [ ]:
twenty_tensor = tf.constant(20, dtype=tf.float64)
five_tensor = tf.constant([5, 2], dtype=tf.float64)
vAvMag = tf.math.pow(tf.constant(10, dtype=tf.float64), tf.math.divide(five_tensor, twenty_tensor))
vAvMag

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([1.77827941, 1.25892541])>

In [1]:
#import numpy as np
#from scipy import signal
#from scipy.fftpack import fft
#from skimage.util import view_as_windows
#from sklearn.decomposition import PCA
import sys
import tensorflow as tf
import tensorflow_probability as tfp


class diff_matrix_estimation(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)

        self.avTimeSec = tf.constant(2.0, dtype=tf.float64)

    def __call__(self, matrix, matrix_shape_, frames_per_second):
        one_tensor = tf.constant(1, dtype=tf.float64)
        twenty_tensor = tf.constant(20, dtype=tf.float64)
        negative_one = tf.constant(-1, dtype=tf.float64)
        factor_exponent = tf.math.divide(tf.math.divide(negative_one, frames_per_second), self.avTimeSec)
        factor = tf.math.subtract(one_tensor, tf.math.exp(factor_exponent))
        newSlow = tf.reshape(matrix[0], (1,10))
        vAvMag = tf.math.pow(tf.constant(10, dtype=tf.float64), tf.math.divide(newSlow, twenty_tensor))
        tf.print('-->>>>>> newSlow --------')
        tf.print(vAvMag[0])

        for f in tf.range(0, matrix_shape_):
            tf.autograph.experimental.set_loop_options(
                shape_invariants=[
                    (newSlow, tf.TensorShape([None, 10])),
                    (vAvMag, tf.TensorShape([None, 10]))
                    ]
            )
            print('Diff Matrix Estimation Eager --------')
            tf.print('Diff Matrix Estimation Graph ---------------------')
            vMag_exponent = tf.math.divide(matrix[f], twenty_tensor)
            vMag = tf.math.pow(tf.constant(10, dtype=tf.float64), vMag_exponent)
            one_minus_factor = tf.math.subtract(one_tensor, factor)
            vAvMag = tf.math.add(tf.math.multiply(vMag, factor), tf.math.multiply(vAvMag, one_minus_factor))
            vAvdB = tf.math.multiply(twenty_tensor, tf.experimental.numpy.log10(vAvMag))

            if tf.math.equal(f, tf.constant(0, dtype=tf.int32)): newSlow = vAvdB
            else: newSlow = tf.concat([newSlow, vAvdB], 0)

        ## Returns newDiff tensor
        return tf.math.subtract(matrix, newSlow)


class windowed_pca(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)
        self.zc1 = tf.Variable(0, trainable=False, dtype=tf.int32)
        self.n = tf.Variable(20, trainable=False, dtype=tf.int32)
        # Define variables for loop
        self.diff_matrix_reshape = tf.Variable([], shape=tf.TensorShape(None), trainable=False, dtype=tf.float64)
        self.principal_components = tf.Variable([], shape=tf.TensorShape(None), trainable=False, dtype=tf.float64)
        # Define window limits
        self.start_window = tf.Variable(0, trainable=False, dtype=tf.int32)
        self.end_window = tf.Variable(0, trainable=False, dtype=tf.int32)
        # Define index for augmented matrix creation
        self.augmented_tensor_idx = tf.Variable(10, trainable=False, dtype=tf.int32)

    def __call__(self, diff_matrix_, matrix_shape_):
        self.zc1.assign(0)
        self.n.assign(20)
        # n = 20 -> 30frames
        ##---------------------> zc1 and zc2 ????
        print('Variables Eager -----')
        tf.print('Variables Graph ---------------------')
        step = tf.constant(1, dtype=tf.int32)

        # Define variables for loop
        self.diff_matrix_reshape.assign([])
        self.principal_components.assign([])

        while tf.less(self.zc1, 3) and tf.greater(self.n, 0):
            # Resets idx for augmented tensor
            self.augmented_tensor_idx.assign(10)
            # Sets new window length
            s = tf.math.floordiv(matrix_shape_, self.n)
            wind0_tensor = self.view_as_windows(diff_matrix_, matrix_shape_, window_shape_0=s, window_shape_1=1, step=step)
            print('Reshape Eager -------------------')
            tf.print('Reshape Graph ---------------------')
            tf.print('s Shape -------------------------')
            tf.print(s)
            tf.print('wind0_tensor Shape -------------------------')
            tf.print(tf.shape(wind0_tensor))
            self.diff_matrix_reshape.assign(wind0_tensor)
            tf.print('self.diff_matrix_reshape Shape -------------------------')
            tf.print(tf.shape(self.diff_matrix_reshape))
            # PCA
            self.principal_components.assign(self.PCA())
            tf.print('self.principal_components Shape -------------------------')
            tf.print(tf.shape(self.principal_components))

            pc1 = self.signal_detrend()
            linspace_num = tf.math.subtract(s, tf.constant(1, dtype=tf.int32))
            linspace_stop = tf.math.subtract(s, tf.constant(1, dtype=tf.int32))
            x = tf.linspace(tf.constant(0, dtype=tf.int32), linspace_stop, linspace_num)

            self.zc1.assign(tf.shape(self.zcr(x, pc1))[0])
            self.n.assign_add(-5)

        return self.principal_components, s, self.diff_matrix_reshape

    def zcr(self, x, y):
        """@brief zero crossing detector"""
        sign_y = tf.math.sign(y)
        tf_diff_axis_1 = tf.math.subtract(sign_y[1:], sign_y[:-1])
        print('Differentiation Eager ---------------')
        tf.print('Differentiation Graph ---------------------')
        not_equal_idx = tf.where(tf.math.not_equal(tf_diff_axis_1, 0))
        x_array = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        x_array = x_array.write(0, x[not_equal_idx[0][0]])
        for idx in tf.range(1, tf.shape(not_equal_idx)[0]):
            x_array = x_array.write(idx, x[not_equal_idx[idx][0]])
        x_array = x_array.stack()

        return x_array

    def view_as_windows(self, diff_matrix_, matrix_shape_, window_shape_0, window_shape_1, step):
        """
        Customized tensorflow version of the skimage function with same name
        """
        wdim = tf.constant(2, dtype=tf.int32)
        zero_tensor = tf.constant(0, dtype=tf.int32)
        one_tensor = tf.constant(1, dtype=tf.int32)
        self.start_window.assign(0)
        self.end_window.assign(window_shape_0)
        print('View as Windows Eager ---------------')
        tf.print('View as Windows Graph ---------------------')

        with tf.control_dependencies([
            tf.debugging.assert_none_equal(wdim, matrix_shape_, message="'window_shape' is incompatible with 'tensor_in.shape'"),
            tf.debugging.assert_greater_equal(step, one_tensor, message="'step' must be >= 1"),
            tf.debugging.assert_none_equal(tf.math.reduce_any(tf.math.less(tf.math.subtract(tf.shape(diff_matrix_)[0], window_shape_0), zero_tensor)),
                                           tf.constant(True), message="'window_shape' is too large"),
            tf.debugging.assert_none_equal(tf.math.reduce_any(tf.math.less(tf.math.subtract(tf.shape(diff_matrix_)[0], one_tensor), zero_tensor)),
                                           tf.constant(True), message="'window_shape' is too small")
            ]):

            augmented_tensor = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
            augmented_tensor = augmented_tensor.write(0, diff_matrix_[self.start_window:self.end_window, 0])
            for i in tf.range(1, 10):
                augmented_tensor = augmented_tensor.write(i, diff_matrix_[self.start_window:self.end_window, i])

            self.start_window.assign_add(step)
            self.end_window.assign_add(step)

            while tf.math.less_equal(self.end_window, matrix_shape_):
                for i in tf.range(0, 10):
                    augmented_tensor = augmented_tensor.write(self.augmented_tensor_idx, diff_matrix_[self.start_window:self.end_window, i])
                    self.augmented_tensor_idx.assign_add(1)

                self.start_window.assign_add(step)
                self.end_window.assign_add(step)

            augmented_tensor = augmented_tensor.stack()

            return augmented_tensor

    def PCA(self):
        print('PCA Eager ------------------')
        tf.print('PCA Graph ---------------------')
        mean_ = tf.math.reduce_mean(self.diff_matrix_reshape, axis=0)
        tensor_input = tf.math.subtract(self.diff_matrix_reshape, mean_)

        #Step-2
        cov_mat = tf.math.multiply(tfp.stats.covariance(tensor_input), tf.constant(2, dtype=tf.float64))

        #Step-3
        eigen_values , eigen_vectors = tf.linalg.eigh(cov_mat)

        #Step-4
        sorted_index = tf.argsort(eigen_values)[::-1]

        #sorted_eigenvalue = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        #sorted_eigenvalue.write(0, eigen_values[sorted_index[0]])
        #for idx in tf.range(1, sorted_index.shape[0]):
        #    sorted_eigenvalue.write(idx, eigen_values[sorted_index[idx]])
        #sorted_eigenvalue = sorted_eigenvalue.stack()

        sorted_eigenvectors = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        sorted_eigenvectors = sorted_eigenvectors.write(0, eigen_vectors[:,sorted_index[0]])
        for idx in tf.range(1, tf.shape(sorted_index)[0]):
            sorted_eigenvectors = sorted_eigenvectors.write(idx, eigen_vectors[:,sorted_index[idx]])
        sorted_eigenvectors = sorted_eigenvectors.stack()

        return sorted_eigenvectors

    def signal_detrend(self):
        print('Signal Detrend Eager -------------')
        tf.print('Signal Detrend Graph ---------------------')
        mean_ = tf.math.reduce_mean(self.principal_components[0], axis=-1, keepdims=True)

        return tf.math.subtract(self.principal_components[0], mean_)


class freq_per_pc(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)
        self.k = tf.Variable(0, trainable=False, dtype=tf.int32)
        # Define index for frequency tensor
        self.freq_pc_idx = tf.Variable(0, trainable=False, dtype=tf.int32)
        """--- Find peaks ---"""
        # Preallocation variables
        self.midpoints = tf.Variable([], shape=tf.TensorShape(None), trainable=False, dtype=tf.int32)
        self.left_edges = tf.Variable([], shape=tf.TensorShape(None), trainable=False, dtype=tf.int32)
        self.right_edges = tf.Variable([], shape=tf.TensorShape(None), trainable=False, dtype=tf.int32)
        # Pointer to the end of valid area in allocated arrays
        self.m = tf.Variable(0, trainable=False, dtype=tf.int32)
        # Pointer to current sample, first one can't be maxima
        self.i = tf.Variable(1, trainable=False, dtype=tf.int32)
        # Define index to look ahead of current sample
        self.i_ahead = tf.Variable(0, trainable=False, dtype=tf.int32)
        """--- Get Harmonic Indices ---"""
        # Define indices for harmonic indices tensor
        self.indh_idx = tf.Variable(0, trainable=False, dtype=tf.int32)
        self.harm = tf.Variable(0, trainable=False, dtype=tf.float64)
        self.harms_idx = tf.Variable(0, trainable=False, dtype=tf.int32)
        self.indf_idx = tf.Variable(0, trainable=False, dtype=tf.int32)
        # Define auxiliar counters
        self.count1 = tf.Variable(2, trainable=False, dtype=tf.float64)
        self.count2 = tf.Variable(2, trainable=False, dtype=tf.float64)
        self.count3 = tf.Variable(0, trainable=False, dtype=tf.int32)
        """--- Add Harmonic Energy ---"""
        # Define index
        self.all_ener_sum_idx = tf.Variable(0, trainable=False, dtype=tf.int32)

    def __call__(self, principal_components_, window_, delta_time_, diff_matrix_, diff_matrix_reshape_):
        self.freq_pc = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)

        tf.print('principal_components_ Shape -------------------------')
        tf.print(tf.shape(principal_components_))
        tf.print('window_ Shape -------------------------')
        tf.print(window_)
        tf.print('diff_matrix_reshape_ Shape -------------------------')
        tf.print(tf.shape(diff_matrix_reshape_))

        for k in tf.range(tf.shape(principal_components_)[0]):
            self.k.assign(k)
            freq_fn, item_in_freq_fn = self.compute_fft(window_, principal_components_, delta_time_)
            indh = self.get_harmonic_indices(freq_fn, item_in_freq_fn)
            rounded_freq_max = self.add_harmonic_energy(freq_fn, indh)

            self.freq_pc = self.freq_pc.write(self.freq_pc_idx, rounded_freq_max)
            self.freq_pc_idx.assign_add(1)
        freq_pc = self.freq_pc.stack()

        #frequencies
        scores = tf.linalg.matmul(diff_matrix_reshape_, principal_components_)

        arr_freq = tf.argsort(tf.math.abs(scores))[:,-1]
        arr_freq_pc_ = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        arr_freq_pc_ = arr_freq_pc_.write(0, freq_pc[arr_freq[0]])
        for idx in tf.range(1, tf.shape(arr_freq)[0]):
            arr_freq_pc_ = arr_freq_pc_.write(idx, freq_pc[arr_freq[idx]])
        arr_freq_pc_ = arr_freq_pc_.stack()

        arr_freq_pc_ = tf.reshape(arr_freq_pc_, (tf.math.floordiv(tf.shape(arr_freq_pc_)[0], 10), 10))
        for _ in tf.range(tf.math.subtract(tf.shape(diff_matrix_)[0], tf.shape(arr_freq_pc_)[0])):
            arr_freq_pc_ = tf.concat([arr_freq_pc_, tf.reshape(arr_freq_pc_[-1,:], tf.TensorShape((1,10)))], 0)

        return arr_freq_pc_

    def compute_fft(self, window_, principal_components_, delta_time_):
        print('Compute Fft Eager -------------------')
        tf.print('Compute Fft Graph ---------------------')
        two_tensor = tf.constant(2.0, dtype=tf.float64)

        # FFT for each PC
        N = tf.math.multiply(window_, 16)
        pc = self.signal_detrend(principal_components_[self.k])
        yf = tf.signal.rfft(pc, [N])

        linspace_num = tf.math.floordiv(N, 2)
        linspace_stop = 1.0 / tf.math.multiply(two_tensor, delta_time_)
        freq = tf.linspace(tf.constant(0, dtype=tf.float64), linspace_stop, linspace_num)

        abs_tensor = tf.math.abs(yf[:linspace_num])
        factor_tensor = two_tensor / tf.cast(N, tf.float64)
        self.PSD = tf.math.multiply(factor_tensor, abs_tensor)

        self.peaks = self.find_peaks(self.PSD)

        freq_fn_ = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        freq_fn_ = freq_fn_.write(0, freq[self.peaks[0]] * 60)
        for idx in tf.range(1, tf.shape(self.peaks)[0]):
            freq_fn_ = freq_fn_.write(idx, freq[self.peaks[idx]] * 60)
        freq_fn_ = freq_fn_.stack()

        self.psd_peaks = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        self.psd_peaks = self.psd_peaks.write(0, self.PSD[self.peaks[0]])
        for idx in tf.range(1, tf.shape(self.peaks)[0]):
            self.psd_peaks = self.psd_peaks.write(idx, self.PSD[self.peaks[idx]])
        self.psd_peaks = self.psd_peaks.stack()

        self.ener_max = tf.math.reduce_max(self.psd_peaks)
        self.freq_max = freq_fn_[tf.where(tf.math.equal(self.psd_peaks, self.ener_max))[0][0]]

        item_in_freq_fn_ = tf.unique(freq_fn_)[0]

        return freq_fn_, item_in_freq_fn_

    def find_peaks(self, psd_tensor):
        print('Find peaks Eager -----------------------')
        tf.print('Find peaks Graph ---------------------')
        #d[tf.Variable(3)].assign(tf.Variable(5))
        # Preallocate, there can't be more maxima than half the size of `x`
        self.midpoints.assign(
            tf.experimental.numpy.empty(
                tf.math.floordiv(tf.shape(psd_tensor)[0], 2),
                dtype=tf.int32
                )
            )
        self.left_edges.assign(
            tf.experimental.numpy.empty(
                tf.math.floordiv(tf.shape(psd_tensor)[0], 2),
                dtype=tf.int32
                )
            )
        self.right_edges.assign(
            tf.experimental.numpy.empty(
                tf.math.floordiv(tf.shape(psd_tensor)[0], 2),
                dtype=tf.int32
                )
            )

        self.m.assign(0)
        self.i.assign(1)
        self.i_ahead.assign(0)

        i_max = tf.math.subtract(tf.shape(psd_tensor)[0], tf.constant(1, dtype=tf.int32))  # Last sample can't be maxima
        while tf.math.less(self.i, i_max):
            # Test if previous sample is smaller
            if tf.math.less(psd_tensor[self.i - 1], psd_tensor[self.i]):
                self.i_ahead.assign(self.i + 1)

                # Find next sample that is unequal to x[i]
                while tf.math.logical_and(
                    tf.math.less(self.i_ahead, i_max),
                    tf.math.equal(psd_tensor[self.i_ahead], psd_tensor[self.i])
                    ):
                    self.i_ahead.assign_add(1)

                # Maxima is found if next unequal sample is smaller than x[i]
                if tf.math.less(psd_tensor[self.i_ahead], psd_tensor[self.i]):
                    self.left_edges[self.m].assign(self.i)
                    self.right_edges[self.m].assign(self.i_ahead - 1)
                    self.midpoints[self.m].assign(tf.math.floordiv((self.left_edges[self.m] + self.right_edges[self.m]), 2))
                    self.m.assign_add(1)
                    # Skip samples that can't be maximum
                    self.i.assign(self.i_ahead)
            self.i.assign_add(1)

        # Keep only valid part of array memory.
        return self.midpoints[:self.m]

    def get_harmonic_indices(self, freq_fn, item_in_freq_fn):
        print('Get Harmonic Indices Eager -------------------')
        tf.print('Get Harmonic Indices Graph ---------------------')

        self.indh_idx.assign(0)

        # Get harmonic indices
        indh = tf.TensorArray(tf.int32, size=0, dynamic_size=True, clear_after_read=False)
        for x in freq_fn:
            self.harm.assign(10)
            harms = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
            self.harms_idx.assign(0)
            self.count1.assign(2)
            self.count2.assign(2)
            while tf.math.less(self.harm, 150):
                if tf.math.less(self.harm, 2):
                    self.harm.assign(tf.math.multiply(x, self.count2))
                    self.count2.assign_add(1)
                    harms = harms.write(self.harms_idx, self.harm)
                    self.harms_idx.assign_add(1)
                else:
                    self.harm.assign(tf.math.divide(x, self.count1))
                    self.count1.assign_add(1)
                    harms = harms.write(self.harms_idx, self.harm)
                    self.harms_idx.assign_add(1)
            harms = harms.stack()
            self.count3.assign(0)
            indf = tf.TensorArray(tf.int32, size=0, dynamic_size=True, clear_after_read=False)
            self.indf_idx.assign(0)
            if tf.reduce_any(tf.math.equal(x, item_in_freq_fn)):
                rounded_x = tf.math.divide(tf.round(tf.math.multiply(x, 100)), 100)
                if tf.math.equal(rounded_x, rounded_x):
                #if round(x,2) in list(map(lambda x: round(x,2), harms)):
                    indf = indf.write(self.indf_idx, self.count3)
                    self.indf_idx.assign_add(1)
                self.count3.assign_add(1)
            indf = indf.stack()
            indh = indh.write(self.indh_idx, indf)
            self.indh_idx.assign_add(1)
        indh = indh.stack()

        return indh

    def add_harmonic_energy(self, freq_fn, indh):
        print('Add Harmonic Energy Eager -------------------------')
        tf.print('Add Harmonic Energy Graph ---------------------')

        self.all_ener_sum_idx.assign(0)

        #Add harmonic energy
        all_ener_sum = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        for ih in tf.range(tf.shape(indh)[0]):
            if tf.math.not_equal(tf.shape(indh[ih])[0], 0):
                psd_peaks_indh = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
                psd_peaks_indh = psd_peaks_indh.write(0, self.psd_peaks[indh[ih][0]])
                for idx in tf.range(1, tf.shape(indh[ih])[0]):
                    psd_peaks_indh = psd_peaks_indh.write(idx, self.psd_peaks[indh[ih][idx]])
                psd_peaks_indh = psd_peaks_indh.stack()
                ener_sum = tf.math.sqrt(tf.math.reduce_sum(tf.math.pow(psd_peaks_indh, 2)))
            else:
                ener_sum = tf.constant(0, dtype=tf.float64)
            all_ener_sum = all_ener_sum.write(self.all_ener_sum_idx, ener_sum)
            self.all_ener_sum_idx.assign_add(1)
        all_ener_sum = all_ener_sum.stack()

        all_ener_sum_max = tf.math.reduce_max(all_ener_sum)
        if tf.math.logical_and(
            tf.math.greater(all_ener_sum_max, self.ener_max),
            tf.math.greater(self.k, 3)
            ):

            all_ener_sum_max_idx = tf.where(tf.math.equal(all_ener_sum, all_ener_sum_max))
            indh_min = tf.TensorArray(tf.int32, size=0, dynamic_size=True, clear_after_read=False)
            indh_min = indh_min.write(0, indh[all_ener_sum_max_idx[0][0]])
            for idx in tf.range(1, tf.shape(all_ener_sum_max_idx)[0]):
                indh_min = indh_min.write(idx, indh[all_ener_sum_max_idx[idx][0]])
            indh_min = indh_min.stack()

            self.freq_max = freq_fn[tf.math.reduce_min(indh_min)] # New freq

        rounded_freq_max = tf.math.divide(tf.round(tf.math.multiply(self.freq_max, 1000)), 1000)

        return rounded_freq_max

    def signal_detrend(self, tensor_in):
        mean_ = tf.math.reduce_mean(tensor_in, axis=-1, keepdims=True)

        return tf.math.subtract(tensor_in, mean_)


class bpm_compute(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)

        self.bpm_idx = tf.Variable(0, trainable=False, dtype=tf.int32)
        self.trust_perc_idx = tf.Variable(0, trainable=False, dtype=tf.int32)
        self.trust_metric_values = self.trust_metric_curve()
        # Define index for resulting frequencies
        self.result_idx = tf.Variable(0, trainable=False, dtype=tf.int32)

    def __call__(self, window_, arr_freq_pc_):
        print('BPM Compute Eager ----------------------')
        tf.print('BPM Compute Graph ---------------------')
        tf.print(tf.shape(arr_freq_pc_))
        frames_ = tf.linspace(tf.constant(0, dtype=tf.int32), tf.shape(arr_freq_pc_)[0], tf.shape(arr_freq_pc_)[0])

        # Define arrays for incremental tensors
        result_freq = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        result_count = tf.TensorArray(tf.int32, size=0, dynamic_size=True, clear_after_read=False)
        bpm_ = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        trust_perc_ = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)

        self.bpm_idx.assign(0)
        self.trust_perc_idx.assign(0)

        for j in tf.range(1, window_):
            band1 = tf.constant(0, dtype=tf.int32)
            a_ = arr_freq_pc_[:j, band1]
            unique_1, _, counts_1 = tf.unique_with_counts(a_, out_idx=tf.int32)

            self.result_idx.assign(0)

            for band2 in tf.range(1,10):
                b_ = arr_freq_pc_[:j, band2]
                unique_2, _, counts_2 = tf.unique_with_counts(b_, out_idx=tf.int32)

                unique_1_2 = tf.concat([unique_1, unique_2], 0)
                unique_concat, _ = tf.unique(unique_1_2, out_idx=tf.int32)
                for key in unique_concat:
                    count_idx_1 = tf.where(tf.math.equal(unique_1, key))
                    count_idx_2 = tf.where(tf.math.equal(unique_2, key))
                    if tf.math.equal(tf.shape(count_idx_1)[0], 0):
                        result_to_write = counts_2[count_idx_2[0][0]]
                    elif tf.math.equal(tf.shape(count_idx_2)[0], 0):
                        result_to_write = counts_1[count_idx_1[0][0]]
                    else:
                        result_to_write = tf.math.add(counts_1[count_idx_1[0][0]], counts_2[count_idx_2[0][0]])
                    result_freq = result_freq.write(self.result_idx, key)
                    result_count = result_count.write(self.result_idx, result_to_write)
                    self.result_idx.assign_add(1)
            result_freq_ = result_freq.stack()
            result_count_ = result_count.stack()

            freq_total_counts = tf.math.reduce_max(result_count_)
            freq_total_idx = tf.where(tf.math.equal(result_count_, freq_total_counts))
            freq_total = result_freq_[freq_total_idx[0][0]]
            perc_ini = tf.math.multiply(tf.math.divide(freq_total_counts, tf.math.multiply(j, 10)), 100)
            perc_ini_to_idx = tf.cast(tf.math.subtract(tf.math.round(perc_ini), 1), dtype=tf.int32)
            perc = self.trust_metric_values[perc_ini_to_idx]

            result_freq = result_freq.unstack(result_freq_)
            result_count = result_count.unstack(result_count_)

            bpm_ = bpm_.write(self.bpm_idx, freq_total)
            trust_perc_ = trust_perc_.write(self.trust_perc_idx, perc)
            self.bpm_idx.assign_add(1)
            self.trust_perc_idx.assign_add(1)
            print('Ya terminé for')

        # Define arrays for incremental tensors
        result_freq2 = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        result_count2 = tf.TensorArray(tf.int32, size=0, dynamic_size=True, clear_after_read=False)
        bpm2_ = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        trust_perc2_ = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)

        for l in tf.range(tf.math.add(tf.math.subtract(tf.shape(arr_freq_pc_)[0], window_), 1)):
            band1 = tf.constant(0, dtype=tf.int32)
            window_add = tf.math.add(l,window_)
            a_ = arr_freq_pc_[l:window_add, band1]
            unique_1, _, counts_1 = tf.unique_with_counts(a_, out_idx=tf.int32)

            self.result_idx.assign(0)

            for band2 in tf.range(1,10):
                b_ = arr_freq_pc_[l:window_add, band2]
                unique_2, _, counts_2 = tf.unique_with_counts(b_, out_idx=tf.int32)

                unique_1_2 = tf.concat([unique_1, unique_2], 0)
                unique_concat, _ = tf.unique(unique_1_2, out_idx=tf.int32)
                for key in unique_concat:
                    count_idx_1 = tf.where(tf.math.equal(unique_1, key))
                    count_idx_2 = tf.where(tf.math.equal(unique_2, key))
                    if tf.math.equal(tf.shape(count_idx_1)[0], 0):
                        result_to_write = counts_2[count_idx_2[0][0]]
                    elif tf.math.equal(tf.shape(count_idx_2)[0], 0):
                        result_to_write = counts_1[count_idx_1[0][0]]
                    else:
                        result_to_write = tf.math.add(counts_1[count_idx_1[0][0]], counts_2[count_idx_2[0][0]])
                    result_freq2 = result_freq2.write(self.result_idx, key)
                    result_count2 = result_count2.write(self.result_idx, result_to_write)
                    self.result_idx.assign_add(1)
            result_freq2_ = result_freq2.stack()
            result_count2_ = result_count2.stack()

            freq_total_counts = tf.math.reduce_max(result_count2_)
            freq_total_idx = tf.where(tf.math.equal(result_count2_, freq_total_counts))
            freq_total = result_freq2_[freq_total_idx[0][0]]
            perc_ini = tf.math.multiply(tf.math.divide(freq_total_counts, tf.math.multiply(window_, 10)), 100)
            perc_ini_to_idx = tf.cast(tf.math.subtract(tf.math.round(perc_ini), 1), dtype=tf.int32)
            perc = self.trust_metric_values[perc_ini_to_idx]

            result_freq2 = result_freq.unstack(result_freq2_)
            result_count2 = result_count.unstack(result_count2_)

            bpm_ = bpm_.write(self.bpm_idx, freq_total)
            trust_perc_ = trust_perc_.write(self.trust_perc_idx, perc)
            self.bpm_idx.assign_add(1)
            self.trust_perc_idx.assign_add(1)

        print('Ya terminé afuera')
        bpm_ = bpm_.stack()
        trust_perc_ = trust_perc_.stack()

        return bpm_, frames_, trust_perc_

    def trust_metric_curve(self):
        tm_values_linspace_1 = tf.linspace(tf.constant(0, dtype=tf.float64), 70, 33)
        tm_values_linspace_2 = tf.linspace(tf.constant(71, dtype=tf.float64), 100, 67)
        tm_values = tf.concat([tm_values_linspace_1, tm_values_linspace_2], 0)

        return tm_values


class RRComputationModule(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)

        self.diff_matrix_estimation = diff_matrix_estimation()
        self.windowed_pca = windowed_pca()
        self.freq_per_pc = freq_per_pc()
        self.bpm_compute = bpm_compute()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, None], dtype=tf.float64),
        tf.TensorSpec(shape=(), dtype=tf.int32),
        tf.TensorSpec(shape=(), dtype=tf.float64),
        tf.TensorSpec(shape=(), dtype=tf.float64),
        tf.TensorSpec(shape=(), dtype=tf.float64)
        ])
    def call(self, matrix, matrix_shape, firmware_version, frames_per_second, delta_time):
        #tf.print(firmware_version)
        print('Main Module ------------------------')
        diff_matrix = self.diff_matrix_estimation(matrix, matrix_shape, frames_per_second)
        #principal_components, window, diff_matrix_reshape = self.windowed_pca(diff_matrix, matrix_shape)
        #arr_freq_pc = self.freq_per_pc(principal_components, window, delta_time, diff_matrix, diff_matrix_reshape)
        #bpm, frames, trust_perc = self.bpm_compute(window, arr_freq_pc)
        return diff_matrix
        #return bpm, frames, trust_perc

In [ ]:
frames_per_second = 10
RR_compute = RRComputationModule(frames_per_second)

matrix_tensor = tf.constant(matrix)

avTimeSec = tf.constant(2.0, dtype=tf.float64)
frames_per_second = tf.constant(frames_per_second, dtype=tf.float64)
factor = 1 - tf.math.exp((-1 / frames_per_second) / avTimeSec)
newSlow = tf.reshape(matrix[0], (1,10))
vAvMag = 10**(newSlow / 20)
for f in range(0, matrix.shape[0]):
    vMag = 10**(matrix[f]/20)
    vAvMag = tf.math.multiply(vMag, factor) + tf.math.multiply(vAvMag, (1 - factor))
    vAvdB = 20 * np.log10(vAvMag)
    if f == 0: newSlow = vAvdB
    else: newSlow = tf.concat([newSlow, vAvdB], 0)

newDiff = tf.math.subtract(matrix, newSlow)

In [ ]:
!pip install fastavro

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 43.4 MB/s eta 0:00:00


In [ ]:
import fastavro
import numpy as np
from RLMeasurement import RLMeasurement
from RLMeasurement_2 import RLMeasurement2



def get_sample(sample_name):
    file_path = '{}.avro'.format(sample_name)
    with open(file_path, 'rb') as avro_file:
        sample_avro_object = fastavro.reader(avro_file)
        sample_avro = [i for i in sample_avro_object][0]

        ## Get sample Hader
        header = sample_avro['Header']
        if 'VersionFirmware' in sample_avro.keys():
            header['firmware_ver'] = sample_avro['VersionFirmware']
            ## Get sample Matrix
            matrix_list = sample_avro['Microphone0']
        else:
            ## Get sample Matrix
            matrix_list = sample_avro['MatrixFast']

        matrix = np.array([matrix_list[0]['band']], dtype=np.float64)
        rows = range(1, len(matrix_list))
        for row in rows:
            matrix = np.append(matrix, np.array([matrix_list[row]['band']], dtype=np.float64), axis=0)

    return matrix, header


def decode_file_frames(sample_name):
    file_path = '{}.bin'.format(sample_name)
    with open(file_path, 'rb') as bytes_file:
        bytes_matrix = bytes_file.read()

    if firmware_version == 1.0:
            measurement = RLMeasurement(bytes_matrix)
            header = measurement.decodeHeader()
            audio_measurement_frames = measurement.decodeAudioFrames(header[6])
    else:
        measurement = RLMeasurement2(bytes_matrix)
        header = measurement.decode_header()
        audio_measurement_frames = measurement.decode_audio_frames(
                                                        header["meas_type"],
                                                        header["frames_dsp_len"])

    return header, measurement, audio_measurement_frames


def get_matrices(firmware_version, measurement, audio_measurement_frames):
        if firmware_version == 1.0:
            _, microphone_0, _ = measurement.preProcessMatrices(
                                                            audio_measurement_frames)
        else:
            microphone_0, _, _ = measurement.pre_process_matrices(
                                                            audio_measurement_frames)

        return microphone_0

## ------------------------------------------------------------------------------------------

In [ ]:
sample_list = [
    #'rl_meas_short_20220325_132257_d41d',
    'rl_meas_short_20220922_154103_dcff_2',
    #'rl_meas_short_20220922_154103_dcff_1',
    #'rl_meas_short_20220922_154103_dcff',
    #'rl_meas_short_20220913_130941_dcff',
    #'rl_meas_short_20220913_130702_dcff',
    #'rl_meas_detailed_20230110_150433_9167',
    #'rl_meas_short_20220325_042658_79b2',
    #'rl_meas_short_20220325_153618_1198',
    #'rl_meas_short_20220318_124449_c5df',
    ]

firmware_version = 1.0

for sample_name in sample_list:
    print(sample_name)
    matrix2, header = get_sample(sample_name)
    header, measurement, bytes_matrix = decode_file_frames(sample_name)
    matrix1 = get_matrices(firmware_version, measurement, bytes_matrix)

    try:
        firmware_version =  header['firmware_ver']
        firmware_version_tensor = tf.constant(header['firmware_ver'], dtype=tf.float64)
    except:
        firmware_version = 1.0
        firmware_version_tensor = tf.constant(1.0, dtype=tf.float64)

    if firmware_version == 1.0:
        frames_per_second = 5
        frames_per_second_tensor = tf.constant(5.0, dtype=tf.float64)
    else:
        frames_per_second = header['dsp_res']
        frames_per_second_tensor = tf.constant(header['dsp_res'], dtype=tf.float64)

    # Tensorflow
    delta_time = 1 / frames_per_second
    delta_time_tensor = tf.math.divide(1, frames_per_second_tensor)
    matrix_tensor = tf.convert_to_tensor(matrix1, dtype=tf.float64)
    matrix_shape_tensor = tf.constant(matrix_tensor.shape[0], dtype=tf.int32)
    print(firmware_version)
    RR_compute_model = RRComputationModule()
    bpm1, frames1, trust_perc1 = RR_compute_model.call(
        matrix_tensor,
        matrix_shape_tensor,
        firmware_version_tensor,
        frames_per_second_tensor,
        delta_time_tensor
        )

    # Python
    bpm2, frames2, trust_perc2 = python_main(
        matrix2,
        frames_per_second
        )

    print(bpm1.numpy()[0])
    print(bpm2[0])

    assert bpm1.numpy() == bpm2
    assert frames1.numpy() == frames2
    assert trust_perc1.numpy() == trust_perc2

In [ ]:
bpm1.numpy()

In [ ]:
tf.config.run_functions_eagerly(False)

In [ ]:
RR_compute_model = RRComputationModule()
#bpm1, frames1, trust_perc1 = RR_compute_model.call(
principal_components = RR_compute_model.call(
        matrix_tensor,
        matrix_shape_tensor,
        firmware_version_tensor,
        frames_per_second_tensor,
        delta_time_tensor
        )

Main Module ------------------------
Diff Matrix Estimation Eager --------
-->>>>>> newSlow --------
[1.0697914634402979 1.0630653160185253 1.0599485062361103 ... 1.0702785577517318 1.0632060898042994 1.0545099688087021]
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Es

In [ ]:
principal_components

<tf.Tensor: shape=(600, 10), dtype=float64, numpy=
array([[ 0.00000000e+00, -6.66133815e-16, -3.33066907e-16, ...,
         2.22044605e-16, -4.44089210e-16, -7.77156117e-16],
       [ 2.37862023e-02,  2.92682961e-02,  2.97841376e-02, ...,
         2.25685327e-02,  2.37150319e-02,  1.83979966e-02],
       [ 4.01463375e-02,  3.43793686e-02,  3.47357337e-02, ...,
         3.31765530e-02,  3.33615243e-02,  3.70986870e-02],
       ...,
       [ 3.53707325e-03,  2.72888994e-03,  1.59002660e-03, ...,
         2.50930533e-03,  1.31839113e-03,  1.15043959e-03],
       [ 2.25127048e-04,  7.81541836e-04,  1.85419083e-03, ...,
        -1.29055269e-03,  4.20614972e-04,  6.07125419e-04],
       [-2.33633529e-03, -1.16057662e-04, -3.47239160e-05, ...,
        -1.23095844e-03, -6.15224044e-04,  6.67135710e-04]])>

In [ ]:
RR_compute_model = RRComputationModule()
tf.saved_model.save(
    RR_compute_model,
    'my_model',
    signatures=RR_compute_model.call.get_concrete_function()
    )

Main Module ------------------------
Diff Matrix Estimation Eager --------
Variables Eager -----
View as Windows Eager ---------------
Reshape Eager -------------------
PCA Eager ------------------
Signal Detrend Eager -------------
Differentiation Eager ---------------
Compute Fft Eager -------------------
Find peaks Eager -----------------------
Get Harmonic Indices Eager -------------------
Add Harmonic Energy Eager -------------------------
BPM Compute Eager ----------------------
Ya terminé for
Ya terminé afuera
Main Module ------------------------
Diff Matrix Estimation Eager --------
Variables Eager -----
View as Windows Eager ---------------
Reshape Eager -------------------
PCA Eager ------------------
Signal Detrend Eager -------------
Differentiation Eager ---------------
Compute Fft Eager -------------------
Find peaks Eager -----------------------
Get Harmonic Indices Eager -------------------
Add Harmonic Energy Eager -------------------------
BPM Compute Eager ----------

In [ ]:
model = tf.saved_model.load('my_model')

In [ ]:
#bpm1, frames1, trust_perc1 = model(
diff_matrix2 = model.call(
        matrix_tensor,
        matrix_shape_tensor,
        firmware_version_tensor,
        frames_per_second_tensor,
        delta_time_tensor
        )

Diff Matrix Estimation Graph ---------------------
View as Windows Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------------------
Diff Matrix Estimation Graph ---------

In [ ]:
diff_matrix2

(<tf.Tensor: shape=(29,), dtype=float64, numpy=
 array([ 48.954, 143.724, 143.724, 143.724, 143.724, 143.724, 143.724,
        143.724, 143.724, 143.724, 143.724, 143.724, 143.724, 143.724,
        143.724, 143.724, 143.724, 143.724, 143.724, 143.724, 143.724,
        143.724, 143.724, 143.724, 143.724, 143.724, 143.724, 143.724,
        143.724])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.])>,
 <tf.Tensor: shape=(29,), dtype=float64, numpy=
 array([41.5625, 19.6875, 26.25  , 30.625 , 32.8125, 35.    , 35.    ,
        37.1875, 37.1875, 37.1875, 37.1875, 37.1875, 37.1875, 39.375 ,
        39.375 , 39.375 , 39.375 , 39.375 , 39.375 , 39.375 , 39.375 ,
        39.375 , 39.375 , 39.375 , 39.375 , 39.375 , 39.375 , 39.375 ,
        39.375 ])>)

In [ ]:
frames_per_second = 10
RR_compute = RRComputationModule(frames_per_second)

matrix_tensor = tf.constant(matrix)
"""diff_matrix = RR_compute.diff_matrix_estimation(matrix)
pca, window, diff_matrix_reshape = RR_compute.windowed_pca(diff_matrix)
arr_freq = RR_compute.freq_per_pc(pca, window.numpy(), diff_matrix, diff_matrix_reshape)
bpm, frames, trust_perc = RR_compute.bpm_compute(window.numpy(), arr_freq)"""
bpm, frames, trust_perc = RR_compute(matrix_tensor)

In [ ]:
frames_per_second = 10
delta_time = 1 / frames_per_second

"""### Compute Respiratory Rate
## Compute difference matrix
diff_matrix = diff_matrix_estimation(matrix, frames_per_second)
## Get pca, window size, and diff matrix with new shape
pca, window, diff_matrix_reshape = window_and_pca(diff_matrix, zc1 = 0, zc2 = 0, n = 20)
## Get frequencies array and window size
arr_freq, window = freq_per_pc(pca, window, delta_time, diff_matrix, diff_matrix_reshape)
## Get bpm, frames, and confidence percentage
bpm, frames, trust_perc = bpm_compute(window, arr_freq)"""
bpm, frames, trust_perc = main(matrix, frames_per_second)

In [ ]:
tf.strings.to_number(tf.strings.split(tf.strings.as_string(tf.timestamp(), precision=2), sep='.')[1], out_type=tf.float64)

<tf.Tensor: shape=(), dtype=float64, numpy=52.0>

In [54]:
import tensorflow as tf

class CppTfTest(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, None], dtype=tf.float32),
        ])
    def call(self, input):

        frames = tf.range(600, dtype=tf.float32)
        """bpm = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
        for idx in frames:
            tf_timestamp = tf.timestamp()
            tf_str_timestamp = tf.strings.as_string(tf_timestamp, precision=2)
            tf_str_split = tf.strings.split(tf_str_timestamp, sep='.')[1]
            tf_split_number = tf.strings.to_number(tf_str_split, out_type=tf.float64)
            bpm = bpm.write(idx, tf_split_number)

        bpm = bpm.stack()"""

        bpm = tf.random.uniform(
            tf.TensorShape([600,]),
            minval=0,
            maxval=90,
            dtype=tf.dtypes.float32
            )

        output = tf.concat(([frames], [bpm]), axis=0)

        return output

In [55]:
cpp_tf_test = CppTfTest()
tf.saved_model.save(
    cpp_tf_test,
    'cpp_tf_test',
    signatures=cpp_tf_test.call.get_concrete_function()
    )

In [56]:
converter = tf.lite.TFLiteConverter.from_saved_model('cpp_tf_test')
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
"""converter.target_spec.experimental_select_user_tf_ops = [
        'TensorArray', 'TensorArrayClose',
        'TensorArrayCloseV2', 'TensorArrayCloseV3',
        'TensorArrayConcat', 'TensorArrayConcatV2',
        'TensorArrayConcatV3', 'TensorArrayGather',
        'TensorArrayGatherV2', 'TensorArrayGatherV3',
        'TensorArrayGrad', 'TensorArrayGradV2',
        'TensorArrayGradV3', 'TensorArrayGradWithShape',
        'TensorArrayPack', 'TensorArrayRead',
        'TensorArrayReadV2', 'TensorArrayReadV3',
        'TensorArrayScatter', 'TensorArrayScatterV2',
        'TensorArrayScatterV3', 'TensorArraySize',
        'TensorArraySizeV2', 'TensorArraySizeV3',
        'TensorArraySplit', 'TensorArraySplitV2',
        'TensorArraySplitV3', 'TensorArrayUnpack',
        'TensorArrayV2', 'TensorArrayV3',
        'TensorArrayWrite', 'TensorArrayWriteV2',
        'TensorArrayWriteV3', 'TensorListConcat',
        'TensorListConcatLists', 'TensorListConcatV2',
        'TensorListElementShape', 'TensorListFromTensor',
        'TensorListGather', 'TensorListGetItem',
        'TensorListLength', 'TensorListPopBack',
        'TensorListPushBack', 'TensorListPushBackBatch',
        'TensorListReserve', 'TensorListResize',
        'TensorListScatter', 'TensorListScatterIntoExistingList',
        'TensorListScatterV2', 'TensorListSetItem',
        'TensorListSplit', 'TensorListStack',
        'AsString', 'Bincount', 'StringSplitV2',
        'StringToNumber', 'Timestamp'
        ]"""
"""converter.target_spec = tf.lite.TargetSpec(
    supported_ops=[tf.lite.OpsSet.TFLITE_BUILTINS],
    experimental_select_user_tf_ops=[
        'RandomUniform', 'Mul'
        ]
)"""
#converter.experimental_new_converter = False
#converter._experimental_lower_tensor_list_ops = False
#converter.signature_def_tensor_names = ["inputs", "outputs"]
#converter.allow_custom_ops=True

tflite_model = converter.convert()

with open('cpp_tf_test.tflite', 'wb') as f:
  f.write(tflite_model)
  f.close()


In [45]:
interpreter = tf.lite.Interpreter('cpp_tf_test.tflite')
#my_signature = interpreter.get_signature_runner()
interpreter.allocate_tensors()

In [9]:
tf.constant([]).numpy()

array([], dtype=float32)

In [46]:
interpreter.invoke()

In [50]:
len(interpreter.get_tensor(8)[0])

TypeError: ignored

In [48]:
interpreter.get_tensor(8)[0]

ValueError: ignored

In [42]:
len(interpreter.get_tensor(8)[1])

600

In [43]:
interpreter.get_tensor(8)[1]

array([5.1580887e+01, 7.0155859e-02, 2.0324257e+01, 7.1548828e+01,
       9.3718319e+00, 2.0837578e+01, 7.7625847e+01, 2.6619497e+01,
       2.4383490e+01, 2.3807953e+01, 6.4083138e+01, 3.8759850e+01,
       1.6617605e+01, 4.8839924e+01, 4.9775478e+01, 5.3193783e+01,
       5.9060097e+00, 2.7454019e-01, 6.9359350e+00, 5.2562275e+01,
       5.0030506e+01, 5.2233994e+01, 4.2054676e+01, 1.0192963e+01,
       3.5740124e+01, 1.5728034e+01, 6.9559547e+01, 2.8398800e+00,
       6.4610191e+01, 8.8572754e+01, 5.9209549e+01, 1.5423818e+01,
       4.8071022e+01, 3.5177773e+01, 1.1406609e+01, 8.4936806e+01,
       5.0323822e+01, 5.6566414e+01, 7.4494690e+01, 5.6069153e+01,
       7.3945969e+01, 8.2011652e+00, 3.4843697e+01, 3.8959816e+01,
       7.2708992e+01, 4.7373764e+01, 2.1024603e+01, 3.2308334e+01,
       7.2814035e+00, 3.8282665e+01, 4.7903053e+01, 6.7908852e+01,
       1.0067286e+01, 9.0857792e+00, 2.9756491e+01, 5.0379524e+01,
       4.3245354e+01, 3.8430252e+01, 8.6714195e+01, 4.0041679e